In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings("ignore")

In [2]:
def rfeFeature(indep_X,dep_Y,n):
        rfelist=[]
        
        log_model =LinearRegression()
        RF = RandomForestRegressor(n_estimators = 10)
        DT= DecisionTreeRegressor(random_state = 0)
        svc_model = SVR(kernel = 'linear', gamma ='scale')
        rfemodellist=[log_model,svc_model,RF,DT] 
        for i in  rfemodellist:
            print(i)
            log_rfe = RFE(i, n)
            log_fit = log_rfe.fit(indep_X, dep_Y)
            log_rfe_feature=log_fit.transform(indep_X)
            rfelist.append(log_rfe_feature)
            #selected_features=indep_X.columns[log_rfe.support_]
            #print("Selected Features",selected_features)
        return rfelist

In [3]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)        
        return X_train, X_test, y_train, y_test

In [4]:
def R2_prediction(regressor,X_test,y_test):
        y_pred = regressor.predict(X_test)
        from sklearn.metrics import r2_score
        r_score=r2_score(y_test,y_pred)        
        return  r_score

In [5]:
def linear(X_train,y_train,X_test):       
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        R2_score=R2_prediction(regressor,X_test,y_test)
        return  regressor,R2_score,X_test,y_test 
    
def svm_linear(X_train,y_train,X_test):
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'linear', C = 1.0)
        regressor.fit(X_train, y_train)
        R2_score=R2_prediction(regressor,X_test,y_test)
        return  regressor,R2_score,X_test,y_test 
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'rbf', C = 1.0)
        regressor.fit(X_train, y_train)
        R2_score=R2_prediction(regressor,X_test,y_test)
        return  regressor,R2_score,X_test,y_test
    
def Decision(X_train,y_train,X_test):
    
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        R2_score=R2_prediction(regressor,X_test,y_test)
        return regressor,R2_score,X_test,y_test     

def random(X_train,y_train,X_test):
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10)
        regressor.fit(X_train, y_train)
        R2_score=R2_prediction(regressor,X_test,y_test)
        return regressor,R2_score,X_test,y_test

In [6]:
def rfe_regressor(acclog,accsvml,accsvmnl,accdes,accrf): 
    
    rfedataframe=pd.DataFrame(index=['Linear','SVR','Random','DecisionTree'],columns=['Linear','SVMl','SVMnl','Decision','Random'])

    for number,idex in enumerate(rfedataframe.index):
        
        rfedataframe['Linear'][idex]=acclog[number]       
        rfedataframe['SVMl'][idex]=accsvml[number]
        rfedataframe['SVMnl'][idex]=accsvmnl[number]
        rfedataframe['Decision'][idex]=accdes[number]
        rfedataframe['Random'][idex]=accrf[number]
    return rfedataframe

In [7]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

In [8]:
df2

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [9]:
indep_X=df2.drop('classification_yes', 1)
dep_Y=df2['classification_yes']

In [10]:
rfelist=rfeFeature(indep_X,dep_Y,7) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                     

In [ ]:
acclog=[]
accsvml=[]
accsvmnl=[]
accdes=[]
accrf=[]

In [ ]:
for i in rfelist:   
    
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)   
            
    regressor,R2_score,X_test,y_test=linear(X_train,y_train,X_test)
    acclog.append(R2_score)
    
    regressor,R2_score,X_test,y_test=svm_linear(X_train,y_train,X_test)
    accsvml.append(R2_score)
    
    regressor,R2_score,X_test,y_test=svm_NL(X_train,y_train,X_test)
    accsvmnl.append(R2_score)
    
    regressor,R2_score,X_test,y_test=Decision(X_train,y_train,X_test)
    accdes.append(R2_score)
    
    regressor,R2_score,X_test,y_test=random(X_train,y_train,X_test)
    accrf.append(R2_score)
    
result=rfe_regressor(acclog,accsvml,accsvmnl,accdes,accrf)

In [ ]:
acclog

In [ ]:
result
#4

In [ ]:
result
#5

In [ ]:
result
#7